# Groupby と Resample 

- 参照
    - [Group by: split-apply-combine — pandas 1.4.1 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#group-by-split-apply-combine)
    - [Resampling — pandas 1.4.1 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling)


## Groupbyとは

1. 1つのデータを複数のグループに分割する(Splitting)
1. 分割した各データに関数を適用して値を得る (Applying)
1. 2で得た値をデータに一つにまとめる  (Combining)


In [84]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    {
        "date": pd.date_range(start="2000-1-1 0:0:0", periods=9, freq="H"),
        "class": np.array(["A", "B", "C"]).repeat(3),
        "value A": np.arange(1,10),
        "value B": np.arange(1,10) * 100,
    }
)

In [85]:
# データ確認
df

,date,class,value A,value B
0,2000-01-01 00:00:00,A,1,100
1,2000-01-01 01:00:00,A,2,200
2,2000-01-01 02:00:00,A,3,300
3,2000-01-01 03:00:00,B,4,400
4,2000-01-01 04:00:00,B,5,500
5,2000-01-01 05:00:00,B,6,600
6,2000-01-01 06:00:00,C,7,700
7,2000-01-01 07:00:00,C,8,800
8,2000-01-01 08:00:00,C,9,900


In [86]:
# class 毎にデータを分割
df_grouped = df.groupby(by="class")
df_grouped


In [87]:
# グループ化した各データの "value A" カラムに max 関数を適用し、一つのデータにまとめる
mx = df_grouped[["value A", "value B"]].max()


In [88]:
# まとめたデータを確認
mx

,value A,value B
class,,
A,3,300
B,6,600
C,9,900


### ポイント

1. dataframe.groupby(by=`"グループ分けしたいcolumn名"`) で、groupby object を得る
1. どの column にたいして、どの関数を適用させるかを決める
    - [GroupBy — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/groupby.html#function-application)

### メモ
- `by=` に渡す column 名は、複数指定可。その場合はリストで渡す。
- 複数の関数を適用させて一度に演算させたい場合は、 `agg` もしくは `aggregate` メソッドを使う
    - [pandas.core.groupby.SeriesGroupBy.aggregate — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.SeriesGroupBy.aggregate.html)
- 自作の関数を使いたい場合は、 `apply` メソッドを使う
    - [pandas.core.groupby.GroupBy.apply — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.apply.html)



### 分割されたデータを確認したい

1. groupby で得られた `groupby object` をリストやループに入れる。ただし巨大なデータは時間がかかる
1. `groupby object` のメソッドである、 `.get_group()` を使う


In [89]:
# 分割されたデータを確認したい
list(df_grouped)

[('A',
                   date class  value A  value B
  0 2000-01-01 00:00:00     A        1      100
  1 2000-01-01 01:00:00     A        2      200
  2 2000-01-01 02:00:00     A        3      300),
 ('B',
                   date class  value A  value B
  3 2000-01-01 03:00:00     B        4      400
  4 2000-01-01 04:00:00     B        5      500
  5 2000-01-01 05:00:00     B        6      600),
 ('C',
                   date class  value A  value B
  6 2000-01-01 06:00:00     C        7      700
  7 2000-01-01 07:00:00     C        8      800
  8 2000-01-01 08:00:00     C        9      900)]

In [90]:
for k, df in df_grouped:
    print(df)

                 date class  value A  value B
0 2000-01-01 00:00:00     A        1      100
1 2000-01-01 01:00:00     A        2      200
2 2000-01-01 02:00:00     A        3      300
                 date class  value A  value B
3 2000-01-01 03:00:00     B        4      400
4 2000-01-01 04:00:00     B        5      500
5 2000-01-01 05:00:00     B        6      600
                 date class  value A  value B
6 2000-01-01 06:00:00     C        7      700
7 2000-01-01 07:00:00     C        8      800
8 2000-01-01 08:00:00     C        9      900


In [91]:
df_grouped.get_group("A")

,date,class,value A,value B
0,2000-01-01 00:00:00,A,1,100
1,2000-01-01 01:00:00,A,2,200
2,2000-01-01 02:00:00,A,3,300


## Resample とは

- "時間"で Groupby すること
- 時系列データの場合はとても良く使う
- まずは以下３つを覚えてください

### 1. 時間を表す index を持つデータに対してのみ使用可
Datetime index や Period index といった、時間を表す index を持つデータに対してのみ使用可

### 2. Frequency String
"Frequency String" と呼ばれる文字列を使って、どの単位時間でデータを分割するかを設定する。参照：[Frequency String](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

例：

時間単位|Frequency String
---|---
日|'D'
時間|'H'
分|'T' or 'min'
秒|'S'
月|'M'
週|'W' 

※週は曜日ではなく、週番号

### 3. `label` オプション

- どちらのエッジでラベルをつけるか



In [92]:
df = pd.DataFrame(
    {
        "date": pd.date_range(start="2000-1-1 0:0:0", periods=9, freq="H"),
        "class": np.array(["A", "B", "C"]).repeat(3),
        "value A": np.arange(1,10),
        "value B": np.arange(1,10) * 100,
    }
)

# date コラムを、このデータのインデックスに設定して上書き
df.set_index("date", inplace=True)
df

,class,value A,value B
date,,,
2000-01-01 00:00:00,A,1,100
2000-01-01 01:00:00,A,2,200
2000-01-01 02:00:00,A,3,300
2000-01-01 03:00:00,B,4,400
2000-01-01 04:00:00,B,5,500
2000-01-01 05:00:00,B,6,600
2000-01-01 06:00:00,C,7,700
2000-01-01 07:00:00,C,8,800
2000-01-01 08:00:00,C,9,900


In [93]:
df.index

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:00:00',
               '2000-01-01 02:00:00', '2000-01-01 03:00:00',
               '2000-01-01 04:00:00', '2000-01-01 05:00:00',
               '2000-01-01 06:00:00', '2000-01-01 07:00:00',
               '2000-01-01 08:00:00'],
              dtype='datetime64[ns]', name='date', freq=None)

In [94]:
# resample() メソッドは、Resampler object を返す
df.resample("2H")

In [95]:
# 2時間毎に、Value Aの max を取得
df.resample("2H")["value A"].max()

date
2000-01-01 00:00:00    2
2000-01-01 02:00:00    4
2000-01-01 04:00:00    6
2000-01-01 06:00:00    8
2000-01-01 08:00:00    9
Freq: 2H, Name: value A, dtype: int64

In [96]:
# 2時間毎に、Value Aの max を取得。ただしラベルは最後のエッジを使用する。
df.resample("2H", label = "right")["value A"].max()


date
2000-01-01 02:00:00    2
2000-01-01 04:00:00    4
2000-01-01 06:00:00    6
2000-01-01 08:00:00    8
2000-01-01 10:00:00    9
Freq: 2H, Name: value A, dtype: int64

## 金融データのResampling

### 取引データからOHLCVを作成
1. 取引データ取得
1. Datetimeindex を持つ DataFrame に変換
1. 時間単位を指定してResampler Objectを作成
1. `.ohlc()` メソッドを適用
1. `.sum()` メソッドを適用（出来高）
1. `.count()` メソッドを適用（取引回数）
1. 表示

In [97]:
import asyncio

import nest_asyncio
import pandas as pd
import plotly.graph_objects as go
import pybotters
from IPython.display import HTML

nest_asyncio.apply()

#### 1. pybotters 経由で FTX の取引データを取得

In [98]:
async def get_trades(market_name, start_time, end_time):
    async with pybotters.Client(
        apis={"ftx": ["", ""]}, base_url="https://ftx.com/api"
    ) as client:
        res = await client.get(
            f"/markets/{market_name}/trades",
            params={
                "start_time": start_time,
                "end_time": end_time,
            },
        )
        return await res.json()

# 取得したデータを確認
data = asyncio.run(get_trades("BTC-PERP", 1643641200, 1643727600))
data["result"][:3]

[{'id': 3254047916,
  'price': 38526.0,
  'size': 0.1464,
  'side': 'buy',
  'liquidation': False,
  'time': '2022-02-01T14:59:59.690649+00:00'},
 {'id': 3254047915,
  'price': 38526.0,
  'size': 0.0536,
  'side': 'buy',
  'liquidation': False,
  'time': '2022-02-01T14:59:59.690649+00:00'},
 {'id': 3254047888,
  'price': 38524.0,
  'size': 0.0047,
  'side': 'sell',
  'liquidation': False,
  'time': '2022-02-01T14:59:59.288778+00:00'}]

#### 2. DataFrame の作成
- `time` コラムを datetimeindex に持つDataFrameを作成

In [99]:
df = pd.DataFrame(data["result"])
df

,id,price,size,side,liquidation,time
0,3254047916,38526.0,0.1464,buy,False,2022-02-01T14:59:59.690649+00:00
1,3254047915,38526.0,0.0536,buy,False,2022-02-01T14:59:59.690649+00:00
2,3254047888,38524.0,0.0047,sell,False,2022-02-01T14:59:59.288778+00:00
3,3254047870,38524.0,0.1210,sell,False,2022-02-01T14:59:59.078000+00:00
4,3254047855,38524.0,0.0127,sell,False,2022-02-01T14:59:58.777561+00:00
...,...,...,...,...,...,...
4995,3253998379,38475.0,0.1500,sell,False,2022-02-01T14:53:19.423013+00:00
4996,3253998377,38475.0,0.0080,sell,False,2022-02-01T14:53:19.370232+00:00
4997,3253998376,38475.0,0.2960,sell,False,2022-02-01T14:53:19.370232+00:00
4998,3253998375,38475.0,0.6960,sell,False,2022-02-01T14:53:19.370232+00:00


In [100]:
df.dtypes


id               int64
price          float64
size           float64
side            object
liquidation       bool
time            object
dtype: object

In [101]:
# time を datetime 型に変更し、この dataframe の index として設定
df = pd.DataFrame(data["result"])
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)
df.sort_index(inplace=True)
df

,id,price,size,side,liquidation
time,,,,,
2022-02-01 14:53:19.339001+00:00,3253998374,38476.0,0.0014,buy,False
2022-02-01 14:53:19.370232+00:00,3253998377,38475.0,0.0080,sell,False
2022-02-01 14:53:19.370232+00:00,3253998375,38475.0,0.6960,sell,False
2022-02-01 14:53:19.370232+00:00,3253998376,38475.0,0.2960,sell,False
2022-02-01 14:53:19.423013+00:00,3253998379,38475.0,0.1500,sell,False
...,...,...,...,...,...
2022-02-01 14:59:58.777561+00:00,3254047855,38524.0,0.0127,sell,False
2022-02-01 14:59:59.078000+00:00,3254047870,38524.0,0.1210,sell,False
2022-02-01 14:59:59.288778+00:00,3254047888,38524.0,0.0047,sell,False


#### 3. resampler object を作成

例：1分足で作成

In [102]:
rule = "1min"
df_resampled = df.resample(rule, label="right")
df_resampled


#### 4. `.ohlc()` メソッドを適用
- resampler オブジェクトのメソッドとして .ohlc() が用意されている
    - [pandas.core.resample.Resampler.ohlc — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.ohlc.html)
- `price` データを OHLC 計算に使えば良い


In [103]:
df_ohlc = df_resampled["price"].ohlc()
df_ohlc

,open,high,low,close
time,,,,
2022-02-01 14:54:00+00:00,38476.0,38538.0,38452.0,38516.0
2022-02-01 14:55:00+00:00,38517.0,38520.0,38383.0,38395.0
2022-02-01 14:56:00+00:00,38395.0,38461.0,38374.0,38417.0
2022-02-01 14:57:00+00:00,38416.0,38417.0,38336.0,38395.0
2022-02-01 14:58:00+00:00,38395.0,38478.0,38394.0,38476.0
2022-02-01 14:59:00+00:00,38472.0,38473.0,38407.0,38473.0
2022-02-01 15:00:00+00:00,38473.0,38550.0,38473.0,38526.0


In [104]:
# 余談ですが、OHLCはこのような形でもできます。

df_ohlc_2 = pd.DataFrame(
    {
        "open": df_resampled["price"].first(),
        "high": df_resampled["price"].max(),
        "low": df_resampled["price"].min(),
        "close": df_resampled["price"].last(),
    }
)

df_ohlc_2

,open,high,low,close
time,,,,
2022-02-01 14:54:00+00:00,38476.0,38538.0,38452.0,38516.0
2022-02-01 14:55:00+00:00,38517.0,38520.0,38383.0,38395.0
2022-02-01 14:56:00+00:00,38395.0,38461.0,38374.0,38417.0
2022-02-01 14:57:00+00:00,38416.0,38417.0,38336.0,38395.0
2022-02-01 14:58:00+00:00,38395.0,38478.0,38394.0,38476.0
2022-02-01 14:59:00+00:00,38472.0,38473.0,38407.0,38473.0
2022-02-01 15:00:00+00:00,38473.0,38550.0,38473.0,38526.0


#### 5. `.sum()` メソッドを適用（出来高）

- size を合計して出来高を出す

In [105]:
df_ohlc["volume"] = df_resampled["size"].sum()
df_ohlc



,open,high,low,close,volume
time,,,,,
2022-02-01 14:54:00+00:00,38476.0,38538.0,38452.0,38516.0,89.7768
2022-02-01 14:55:00+00:00,38517.0,38520.0,38383.0,38395.0,201.3913
2022-02-01 14:56:00+00:00,38395.0,38461.0,38374.0,38417.0,257.0133
2022-02-01 14:57:00+00:00,38416.0,38417.0,38336.0,38395.0,213.7127
2022-02-01 14:58:00+00:00,38395.0,38478.0,38394.0,38476.0,177.0822
2022-02-01 14:59:00+00:00,38472.0,38473.0,38407.0,38473.0,94.0939
2022-02-01 15:00:00+00:00,38473.0,38550.0,38473.0,38526.0,123.6707


#### 6. `.count()` メソッドを適用（取引回数）

- `id` を数えて取引回数を出す。
- 行数のカウントなので `id` 以外でもよい

In [106]:
df_ohlc["count"] = df_resampled["id"].count()
df_ohlc

,open,high,low,close,volume,count
time,,,,,,
2022-02-01 14:54:00+00:00,38476.0,38538.0,38452.0,38516.0,89.7768,410
2022-02-01 14:55:00+00:00,38517.0,38520.0,38383.0,38395.0,201.3913,739
2022-02-01 14:56:00+00:00,38395.0,38461.0,38374.0,38417.0,257.0133,1097
2022-02-01 14:57:00+00:00,38416.0,38417.0,38336.0,38395.0,213.7127,939
2022-02-01 14:58:00+00:00,38395.0,38478.0,38394.0,38476.0,177.0822,763
2022-02-01 14:59:00+00:00,38472.0,38473.0,38407.0,38473.0,94.0939,451
2022-02-01 15:00:00+00:00,38473.0,38550.0,38473.0,38526.0,123.6707,601


#### 関数化



In [107]:
def generate_ohlcv(df_resampled):
    df_ohlc = df_resampled["price"].ohlc()
    df_ohlc["volume"] = df_resampled["size"].sum()
    df_ohlc["count"] = df_resampled["id"].count()
    return df_ohlc



### OHLCV を sell と buy で分けて作成
- FTX から取得したデータに `side` があるので、`side` で groupby して、resample する

In [108]:
rule = "10s"
df_buy_resampled = df.groupby("side").get_group("buy").resample(rule, label="right")
df_sell_resampled = df.groupby("side").get_group("sell").resample(rule, label="right")
df_buy = generate_ohlcv(df_buy_resampled)
df_sell = generate_ohlcv(df_sell_resampled)

In [109]:
# コラム名をリネーム
df_buy.rename(columns={c:f"{c}_buy" for c in df_buy.columns}, inplace=True)
df_sell.rename(columns={c:f"{c}_sell" for c in df_sell.columns}, inplace=True)


In [110]:
# DataFrame のConcat
pd.concat([df_buy,df_sell], axis=1)

,open_buy,high_buy,low_buy,close_buy,volume_buy,count_buy,open_sell,high_sell,low_sell,close_sell,volume_sell,count_sell
time,,,,,,,,,,,,
2022-02-01 14:53:20+00:00,38476.0,38482.0,38476.0,38482.0,0.4005,4,38475.0,38485.0,38475.0,38484.0,2.2944,9
2022-02-01 14:53:30+00:00,38483.0,38493.0,38481.0,38493.0,2.4498,27,38477.0,38492.0,38476.0,38492.0,4.4507,22
2022-02-01 14:53:40+00:00,38489.0,38493.0,38453.0,38493.0,10.7624,49,38492.0,38494.0,38452.0,38494.0,27.6822,106
2022-02-01 14:53:50+00:00,38495.0,38508.0,38491.0,38508.0,8.8073,35,38494.0,38508.0,38490.0,38504.0,4.1168,32
2022-02-01 14:54:00+00:00,38505.0,38536.0,38504.0,38517.0,6.8231,44,38504.0,38538.0,38503.0,38516.0,21.9896,82
2022-02-01 14:54:10+00:00,38517.0,38517.0,38480.0,38480.0,0.9145,17,38520.0,38520.0,38459.0,38468.0,33.9900,113
2022-02-01 14:54:20+00:00,38468.0,38482.0,38452.0,38477.0,26.8089,81,38472.0,38479.0,38452.0,38479.0,20.7352,69
2022-02-01 14:54:30+00:00,38480.0,38484.0,38480.0,38482.0,3.0599,20,38479.0,38483.0,38479.0,38482.0,2.4673,10
2022-02-01 14:54:40+00:00,38482.0,38494.0,38460.0,38461.0,5.7062,25,38481.0,38493.0,38459.0,38463.0,11.6122,50


## アップサンプリングとダウンサンプリング

- ダウンサンプリング ：高頻度から低頻度へ（毎日→毎月）
- アップサンプリング ：低頻度から高頻度へ（毎週→毎日）
- 今日話した内容は全てダウンサンプリング。
- アップサンプリングしたい場合も同様に可。データがない場合は NaNが返る。


In [111]:
# 例：1分足で作った df_ohlc の close データを使って 30秒の max を得る
df_ohlc.resample("30s")["close"].max()

time
2022-02-01 14:54:00+00:00    38516.0
2022-02-01 14:54:30+00:00        NaN
2022-02-01 14:55:00+00:00    38395.0
2022-02-01 14:55:30+00:00        NaN
2022-02-01 14:56:00+00:00    38417.0
2022-02-01 14:56:30+00:00        NaN
2022-02-01 14:57:00+00:00    38395.0
2022-02-01 14:57:30+00:00        NaN
2022-02-01 14:58:00+00:00    38476.0
2022-02-01 14:58:30+00:00        NaN
2022-02-01 14:59:00+00:00    38473.0
2022-02-01 14:59:30+00:00        NaN
2022-02-01 15:00:00+00:00    38526.0
Freq: 30S, Name: close, dtype: float64

## ちょっと tips 

### 曜日毎にグループ化したい場合は resample ではなく groupby 
- `index.strftime("%w")` で曜日番号、もしくは `index.strftime("%a")` で曜日文字列を得て、カラムに追加。そのカラムで groupby する
- `.resample("w")` は 週番号

### 各国祝祭日を得たいとき
- [pandas-market-calendars · PyPI](https://pypi.org/project/pandas-market-calendars/)
